In [1]:
using PyCall
using JuMP
using Ipopt
unshift!(PyVector(pyimport("sys")["path"]), "");

@pyimport matplotlib.pyplot as plt
@pyimport numpy as np
@pyimport json
@pyimport os
@pyimport pickle
@pyimport pandas as pd
@pyimport collections
@pyimport utils_julia
@pyimport parameters_julia
@pyimport OD_functions
@pyimport runGLS

out_dir = parameters_julia.out_dir
files_ID = parameters_julia.files_ID
month_w = parameters_julia.month_w
year = parameters_julia.year
instances_1 = parameters_julia.instances_ID
deg_grid = parameters_julia.deg_grid
c_grid = parameters_julia.c_grid
lamb_grid = parameters_julia.lamb_grid
week_day_Apr_list = parameters_julia.week_day_list
nzones = parameters_julia.n_zones
average_over_time = parameters_julia.average_over_time_p
time_instances = parameters_julia.time_instances

if isdir(out_dir * "OD_demands") == false
    mkdir(out_dir * "OD_demands")
end

#for day_1 in week_day_Apr_list
day_1 = 6
    inst = instances_1[1]
    open(out_dir * "instance_comm.txt", "w") do f
        write(f, inst)
    end

    #day_1 = week_day_Apr_list[1]
    open(out_dir * "day_comm.txt", "w") do g
        write(g, string(day_1))
    end

    A = runGLS.runGLS_f(out_dir, files_ID, time_instances, month_w, week_day_Apr_list, average_over_time)
    xi_list = A[1]
    P = A[2]
    L = A[3]
    edges = A[4]
    
    
    #xi_list = runGLS.xi_list
    #P = runGLS.P
    #L = runGLS.L  # dimension of xi
    mGLSJulia = []
    lam = []
    lam_list = []
    route_choice_matrix_Apr_AM = []
    p = []
    
    
    mGLSJulia = Model(solver=IpoptSolver())

    @variable(mGLSJulia, lam[1:size(P,1)] >= 0)

    @variable(mGLSJulia, p[1:size(P,1), 1:size(P,2)] >= 0)

    for i = 1:size(P,1)
        for j = 1:size(P,2)
            if P[i,j] == 0
                @constraint(mGLSJulia, p[i,j] == 0)
            end
        end
    end

    for i = 1:size(P,1)
        @constraint(mGLSJulia, sum(p[i,j] for j = 1:size(P,2)) == 1)
    end

    for l = 1:L
        @constraint(mGLSJulia, sum(p[i,l] * lam[i] for i = 1:size(P,1)) == xi_list[l])
    end

    @objective(mGLSJulia, Min, sum(p[1,j] for j = 1:size(P,2)))  # play no actual role, but could not use zero objective

    solve(mGLSJulia)

    #getvalue(lam)

    getobjectivevalue(mGLSJulia)


    #edges = runGLS.edges
    edges = length(edges)
    lam_list = getvalue(lam)
    runGLS.saveDemandVec(nzones, out_dir, inst, files_ID, lam_list, month_w, string(day_1) )


    #println(lam)


    getvalue(p)

    route_choice_matrix_Apr_AM = getvalue(p)

    sum(route_choice_matrix_Apr_AM[1, :])

    route_choice_matrix_dict_Apr_AM = Dict()

    for row_idx = 1:size(route_choice_matrix_Apr_AM)[1]
        for col_idx = 1:size(route_choice_matrix_Apr_AM)[2]
            route_choice_matrix_dict_Apr_AM[(row_idx, col_idx)] = route_choice_matrix_Apr_AM[row_idx, col_idx]
        end
    end


    outfile = open("route_choice" * files_ID * "_" * month_w * "_" * inst * ".json", "w")

    JSON.print(outfile, route_choice_matrix_dict_Apr_AM)

    close(outfile)

    getvalue(p)

    route_choice_matrix_Apr_PM = getvalue(p)

    sum(route_choice_matrix_Apr_PM[1, :])

    route_choice_matrix_dict_Apr_PM = Dict()

    for row_idx = 1:size(route_choice_matrix_Apr_PM)[1]
        for col_idx = 1:size(route_choice_matrix_Apr_PM)[2]
            route_choice_matrix_dict_Apr_PM[(row_idx, col_idx)] = route_choice_matrix_Apr_PM[row_idx, col_idx]
        end
    end

    outfile = open(out_dir * "OD_demands/" * "route_choice"* files_ID * "_" * month_w * "_" * string(day_1) * "_" * inst * ".json", "w")

    JSON.print(outfile, route_choice_matrix_dict_Apr_PM)

    close(outfile)

#end

Academic license - for non-commercial use only

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    18286
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     4592

Total number of variables............................:     4674
                     variables with only lower bounds:     4674
                variables with lower and upper bounds:        0
                     variables with o